In [123]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [124]:
import numpy as np

class Router:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.neighbors = [] 
        self.edges = []  

class Edge:
    def __init__(self, start, end, length):
        self.start = start
        self.end = end
        self.length = length
        self.load = 0  



# sample dataset 

In [125]:


# def generate_sample_data(m, n, num_samples):
#     data = {
#         'Source': [],
#         'Destination': [],
#         'Current_Router': [],
#         'Action': [],
#         'Next_Router': []
#     }

#     for _ in range(num_samples):
#         source = (random.randint(0, m-1), random.randint(0, n-1))
#         destination = (random.randint(0, m-1), random.randint(0, n-1))
#         current_router = source
#         action = random.choice(['down', 'up', 'left', 'right'])

#         if action == 'down' and current_router[0] < destination[0]:
#             next_router = (current_router[0] + 1, current_router[1])
#         elif action == 'up' and current_router[0] > destination[0]:
#             next_router = (current_router[0] - 1, current_router[1])
#         elif action == 'right' and current_router[1] < destination[1]:
#             next_router = (current_router[0], current_router[1] + 1)
#         elif action == 'left' and current_router[1] > destination[1]:
#             next_router = (current_router[0], current_router[1] - 1)
#         else:
#             next_router = current_router  

#         data['Source'].append(source)
#         data['Destination'].append(destination)
#         data['Current_Router'].append(current_router)
#         data['Action'].append(action)
#         data['Next_Router'].append(next_router)

#     return pd.DataFrame(data)
def generate_sample_data(m, n, num_samples):
    data = {
        'Source': [],
        'Destination': [],
        'Current_Router': [],
        'Up': [],
        'Right': [],
        'Down': [],
        'Left': [],
        'Next_Router': []
    }

    for _ in range(num_samples):
        source = (random.randint(0, m-1), random.randint(0, n-1))
        destination = (random.randint(0, m-1), random.randint(0, n-1))
        current_router = (random.randint(0, m-1), random.randint(0, n-1))
        action = random.choice(['down', 'up', 'left', 'right'])
        up, down, left, right = 0, 0, 0, 0
        if(action=='up'):up=1
        elif(action=='down'):down=1
        elif(action=='left'):left=1
        elif(action=='right'):right=1


        next_router = current_router  # Default to staying in place if action not possible
        if down==1 and current_router[0] < m-1:
            next_router = (current_router[0] + 1, current_router[1])
        elif up==1 and current_router[0] > 0:
            next_router = (current_router[0] - 1, current_router[1])
        elif right==1 and current_router[1] < n-1:
            next_router = (current_router[0], current_router[1] + 1)
        elif left==1 and current_router[1] > 0:
            next_router = (current_router[0], current_router[1] - 1)

        # Add the generated data to the dictionary
        data['Source'].append(source)
        data['Destination'].append(destination)
        data['Current_Router'].append(current_router)
        data['Up'].append(up)
        data['Down'].append(down)
        data['Left'].append(left)
        data['Right'].append(right)
        data['Next_Router'].append(next_router)

    # Convert the dictionary to a DataFrame and return
    return pd.DataFrame(data)


def observation(df):
    observations_data = {
        'Source_X': [],
        'Source_Y': [],
        'Destination_X': [],
        'Destination_Y': [],
        'Current_X': [],
        'Current_Y': [],
        'Up': [],
        'Right': [],
        'Down': [],
        'Left': [],
        'Next_X': [],
        'Next_Y': []
    }

    for index, row in df.iterrows():
        source_x, source_y = row['Source']
        dest_x, dest_y = row['Destination']
        current_x, current_y = row['Current_Router']
        up=row['Up']
        down=row['Down']
        left=row['Left']
        right=row['Right']
        next_x, next_y = row['Next_Router'] 

        observations_data['Source_X'].append(source_x)
        observations_data['Source_Y'].append(source_y)
        observations_data['Destination_X'].append(dest_x)
        observations_data['Destination_Y'].append(dest_y)
        observations_data['Current_X'].append(current_x)
        observations_data['Current_Y'].append(current_y)
        observations_data['Up'].append(up)
        observations_data['Down'].append(down)
        observations_data['Left'].append(left)
        observations_data['Right'].append(right)
        observations_data['Next_X'].append(next_x)
        observations_data['Next_Y'].append(next_y)

    return pd.DataFrame(observations_data)


m, n = 5, 5  
num_samples = 20

sample_data = generate_sample_data(m, n, num_samples)

observations_df = observation(sample_data)

# action_mapping = {'up': 0, 'right': 1, 'down': 2, 'left': 3}
# observations_df['Action'] = observations_df['Action'].map(action_mapping)

print(observations_df)


    Source_X  Source_Y  Destination_X  Destination_Y  Current_X  Current_Y  \
0          0         1              4              0          4          4   
1          3         1              3              0          2          2   
2          4         4              1              2          1          1   
3          4         3              1              0          4          2   
4          1         1              2              0          1          4   
5          4         3              1              3          1          3   
6          3         4              0              0          4          4   
7          0         3              1              1          2          0   
8          4         3              1              3          2          4   
9          2         1              1              0          3          4   
10         3         3              4              0          2          3   
11         3         4              4              1          4 

In [126]:
sample_data.head()

,Source,Destination,Current_Router,Up,Right,Down,Left,Next_Router
0,"(0, 1)","(4, 0)","(4, 4)",1,0,0,0,"(3, 4)"
1,"(3, 1)","(3, 0)","(2, 2)",0,1,0,0,"(2, 3)"
2,"(4, 4)","(1, 2)","(1, 1)",1,0,0,0,"(0, 1)"
3,"(4, 3)","(1, 0)","(4, 2)",0,0,0,1,"(4, 1)"
4,"(1, 1)","(2, 0)","(1, 4)",0,0,1,0,"(2, 4)"


# Defining Gan

In [127]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )
        
        self.action_head = nn.Sequential(
            nn.Linear(output_dim, 4), 
            nn.Softmax(dim=-1)
        )
        
    def forward(self, x):
        main_output = self.main(x)
        action_probs = self.action_head(main_output)
        return main_output, action_probs  

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3), 
            nn.Linear(128, 1),
            nn.Sigmoid()  
        )
        
    def forward(self, x):
        return self.net(x)




## Making a dataloader for the gan from our dataset


In [128]:

observations_np = observations_df.to_numpy()
observations_tensor = torch.tensor(observations_np, dtype=torch.float64)

from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(observations_tensor)

batch_size = 64  
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



In [129]:

noise_vector_size = 100  

noise = torch.randn(batch_size, noise_vector_size, device= torch.device("cuda" if torch.cuda.is_available() else "cpu"))


In [130]:
def train_gan(generator, discriminator, gen_optimizer, disc_optimizer, real_data_loader, epochs, device):
    criterion = nn.BCELoss()
    
    for epoch in range(epochs):
        for real_data in real_data_loader:
            real_data = real_data[0].to(device) 
            real_data = real_data.float()
            if real_data.shape[1] != 12:
                raise ValueError(f'Expected {12} features, but got {real_data.shape[1]}')
            batch_size = real_data.size(0)


            disc_optimizer.zero_grad()


            real_labels = torch.ones(batch_size, 1, device=device)


            real_output = discriminator(real_data)
            real_loss = criterion(real_output, real_labels)


            noise = torch.randn(batch_size, noise_vector_size, device=device)  
            fake_data, _ = generator(noise)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            fake_output = discriminator(fake_data.detach())
            fake_loss = criterion(fake_output, fake_labels)


            disc_loss = real_loss + fake_loss
            disc_loss.backward()
            disc_optimizer.step()


            gen_optimizer.zero_grad()
            trick_labels = torch.ones(batch_size, 1, device=device)
            trick_output = discriminator(fake_data)
            gen_loss = criterion(trick_output, trick_labels)
            gen_loss.backward()
            gen_optimizer.step()

        if epoch % 10 == 0: 
            print(f'Epoch [{epoch+1}/{epochs}], Loss D: {disc_loss.item()}, Loss G: {gen_loss.item()}')


In [131]:
import torch
import torch.optim as optim

noise_dim = 100
gen_output_dim = 12
disc_input_dim = gen_output_dim  


generator = Generator(input_dim=noise_dim, output_dim=gen_output_dim)
discriminator = Discriminator(input_dim=disc_input_dim)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)


gen_optimizer = optim.Adam(generator.parameters(), lr=0.002)
disc_optimizer = optim.Adam(discriminator.parameters(), lr=0.002)


In [132]:
epochs = 1000  

train_gan(
    generator=generator, 
    discriminator=discriminator, 
    gen_optimizer=gen_optimizer, 
    disc_optimizer=disc_optimizer, 
    real_data_loader=data_loader, 
    epochs=epochs, 
    device=device
)


Epoch [1/1000], Loss D: 1.4574804306030273, Loss G: 0.6436331868171692
Epoch [11/1000], Loss D: 1.0226306915283203, Loss G: 0.7691770792007446
Epoch [21/1000], Loss D: 0.9784233570098877, Loss G: 0.8901458978652954
Epoch [31/1000], Loss D: 0.1612567901611328, Loss G: 2.6684887409210205
Epoch [41/1000], Loss D: 0.05125553905963898, Loss G: 4.738255023956299
Epoch [51/1000], Loss D: 0.024436943233013153, Loss G: 5.415875434875488
Epoch [61/1000], Loss D: 1.0855342149734497, Loss G: 2.1304662227630615
Epoch [71/1000], Loss D: 0.02091560885310173, Loss G: 5.183468818664551
Epoch [81/1000], Loss D: 0.1203194260597229, Loss G: 4.935229778289795
Epoch [91/1000], Loss D: 0.006096262019127607, Loss G: 7.385222434997559
Epoch [101/1000], Loss D: 0.06759977340698242, Loss G: 6.984364986419678
Epoch [111/1000], Loss D: 0.002173712942749262, Loss G: 9.563599586486816
Epoch [121/1000], Loss D: 0.0027347782161086798, Loss G: 9.362735748291016
Epoch [131/1000], Loss D: 0.011093508452177048, Loss G: 7.

## Viewing the gan's synthetic data

In [133]:

noise_vector_size = 100
batch_size = 100  

noise = torch.randn(batch_size, noise_vector_size, device=device)

with torch.no_grad():
    generated_data,_ = generator(noise)
    
df_generated = pd.DataFrame(generated_data.cpu().numpy())
print(df_generated.head(100))  


          0         1         2         3         4         5         6   \
0  -0.930242  0.999999  0.999922  0.999997  0.999936  0.999429  0.279414   
1   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000 -0.417433   
2  -0.918043  0.999998  0.999909  0.999994  0.999919  0.999478  0.266755   
3   0.999417  1.000000  1.000000  1.000000  0.999999  0.999999 -0.370797   
4  -0.944955  1.000000  0.999967  0.999999  0.999976  0.999536  0.316767   
..       ...       ...       ...       ...       ...       ...       ...   
95 -0.904201  0.999997  0.999888  0.999996  0.999918  0.998580  0.304325   
96  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000 -0.314103   
97 -0.795170  0.999990  0.999700  0.999981  0.999726  0.995205  0.215226   
98 -0.419752  0.999992  0.999443  0.999966  0.999584  0.992947  0.074180   
99 -0.602166  0.999991  0.999713  0.999950  0.999529  0.995830  0.221768   

          7         8         9         10        11  
0   0.160044 -0.333024 -0.233779

In [136]:
import pandas as pd
import numpy as np
import torch

def decode_gan_output(gan_output, mesh_size_x, mesh_size_y):
    action_mapping = {0: "up", 1: "right", 2: "down", 3: "left"}
    decoded_data = []
    for row in gan_output:
        source_x = int((row[0] + 1) * mesh_size_x / 2)%mesh_size_x
        source_y = int((row[1] + 1) * mesh_size_y / 2)%mesh_size_y
        destination_x = int((row[2] + 1) * mesh_size_x / 2)%mesh_size_x
        destination_y = int((row[3] + 1) * mesh_size_y / 2)%mesh_size_y
        current_x = int((row[4] + 1) * mesh_size_x / 2)%mesh_size_x
        current_y = int((row[5] + 1) * mesh_size_y / 2)%mesh_size_y

        action_probs = torch.tensor((row[6:10] + 1) / 2)


        action_idx = torch.argmax(action_probs).item()
        action = action_mapping[action_idx]
        next_x = int((row[10] + 1) * mesh_size_x)%mesh_size_x
        next_y = int((row[11] + 1) * mesh_size_y)%mesh_size_y
        decoded_data.append([source_x, source_y, destination_x, destination_y, current_x, current_y, action, next_x, next_y])
    return pd.DataFrame(decoded_data, columns=['Source_X', 'Source_Y', 'Destination_X', 'Destination_Y', 'Current_X', 'Current_Y', 'Action', 'Next_X', 'Next_Y'])


df_generated_values = df_generated.to_numpy()


decoded_df = decode_gan_output(df_generated_values, 5, 5)
print(decoded_df.head(20))


    Source_X  Source_Y  Destination_X  Destination_Y  Current_X  Current_Y  \
0          0         4              4              4          4          4   
1          0         0              0              0          0          0   
2          0         4              4              4          4          4   
3          4         0              4              0          4          4   
4          0         4              4              4          4          4   
5          4         0              0              0          0          0   
6          0         4              4              4          4          4   
7          0         0              0              0          0          0   
8          0         4              4              4          4          4   
9          0         4              4              4          4          4   
10         0         4              4              4          4          4   
11         4         0              0              0          0 

# Q-Learning

## Converting GAN's data to state-action-reward-next-state format

In [39]:
import pandas as pd
import numpy as np

def convert_to_q_learning_format(df, reward_per_step=-1, reward_for_success=10):
    q_data = []

    for index, row in df.iterrows():
        state = (row['Current_X'], row['Current_Y'], row['Destination_X'], row['Destination_Y'])
        action_map = {'up': 0, 'down': 1, 'left': 2, 'right': 3}
        action = action_map[row['Action']]
        

        if (row['Next_X'], row['Next_Y']) == (row['Destination_X'], row['Destination_Y']):
            reward = reward_for_success  
        else:
            reward = reward_per_step 
        

        next_state = (row['Next_X'], row['Next_Y'], row['Destination_X'], row['Destination_Y'])
        
        q_data.append((state, action, reward, next_state))
    
    return q_data


df = pd.DataFrame(decoded_df)  
q_learning_data = convert_to_q_learning_format(df)


In [49]:
import numpy as np

m, n = 5, 5 
n_actions = 4  
learning_rate = 0.1
discount_factor = 0.99

n_states = m * n * m * n

Q_table = np.zeros((n_states, n_actions))

def get_state_index(state, m, n):
    current_x, current_y, destination_x, destination_y = state
    return (current_x * n + current_y) * (m * n) + (destination_x * n + destination_y)


q_learning_data = q_learning_data


for state, action, reward, next_state in q_learning_data:
    state_index = get_state_index(state, m, n)
    next_state_index = get_state_index(next_state, m, n)
    
    current_q_value = Q_table[state_index, action]
    print(next_state)
    next_max_q = np.max(Q_table[next_state_index])
    

    new_q_value = current_q_value + learning_rate * (reward + discount_factor * next_max_q - current_q_value)
    Q_table[state_index, action] = new_q_value
    
    


(4, 4, 4, 4)
(5, 5, 5, 5)


IndexError: index 780 is out of bounds for axis 0 with size 625

In [ ]:
import pandas as pd
import numpy as np

n_states = 100 
n_actions = 4  



Q_table_df = pd.DataFrame(Q_table, columns=['Up', 'Down', 'Left', 'Right'])

# Print the DataFrame
print(Q_table_df)


          Up      Down      Left     Right
0   0.028364  0.758772  0.598829  0.974205
1   0.166743  0.688909  0.640527  0.975692
2   0.202374  0.381874  0.275995  0.940433
3   0.765112  0.994133  0.016940  0.383205
4   0.340881  0.906992  0.789492  0.928716
..       ...       ...       ...       ...
95  0.432117  0.384925  0.192939  0.701388
96  0.378212  0.060167  0.363165  0.383217
97  0.648298  0.772494  0.367362  0.467915
98  0.087737  0.730701  0.357271  0.145272
99  0.848722  0.908848  0.394579  0.912282

[100 rows x 4 columns]


In [ ]:
def state_to_index(state, size_x):
    x, y = state
    return x * size_x + y

def index_to_state(index, size_x):
    x = index // size_x
    y = index % size_x
    return (x, y)


In [ ]:
size_x = 5  


state = (2, 3) 
index = state_to_index(state, size_x)
print(f"State {state} corresponds to index {index}")


index = 13
state = index_to_state(index, size_x)
print(f"Index {index} corresponds to state {state}")


State (2, 3) corresponds to index 13
Index 13 corresponds to state (2, 3)


In [ ]:
def get_optimal_path(source, destination, Q_table, state_to_index, index_to_state, size_x):

    current_state = source
    path = [current_state]

    while current_state != destination:
        current_index = state_to_index(current_state, size_x)
        action = np.argmax(Q_table[current_index])  
        

        if action == 0:  
            next_state = (current_state[0] - 1, current_state[1])
        elif action == 1:  
            next_state = (current_state[0], current_state[1] + 1)
        elif action == 2:  
            next_state = (current_state[0] + 1, current_state[1])
        else:  
            next_state = (current_state[0], current_state[1] - 1)
        
        path.append(next_state)
        current_state = next_state


        if len(path) > size_x * size_x or path.count(current_state) > 1:
            break

    return path


test_pairs = [((0, 0), (4, 4)), ((2, 3), (1, 1))]

for source, destination in test_pairs:
    optimal_path = get_optimal_path(source, destination, Q_table, state_to_index, index_to_state, size_x=5)
    print(f"Optimal path from {source} to {destination}: {optimal_path}")


Optimal path from (0, 0) to (4, 4): [(0, 0), (0, -1), (0, -2), (0, -1)]
Optimal path from (2, 3) to (1, 1): [(2, 3), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0), (0, -1), (0, -2), (0, -1)]


In [ ]:
# Example test pairs using coordinates in a 5x5 mesh
test_pairs = [
    {'source': (0, 0), 'destination': (4, 4)},  # From top-left to bottom-right
    {'source': (2, 3), 'destination': (1, 1)},  # From one point to another within the mesh
    # Add more pairs as needed
]

# Convert to your model's input format if necessary
# For example, if your model uses flattened indices instead of (x, y) coordinates
def convert_coordinates_to_index(x, y, size_x):
    return x * size_x + y

# Example of converting a test pair to the model's expected input format
for pair in test_pairs:
    source_index = convert_coordinates_to_index(pair['source'][0], pair['source'][1], size_x=5)
    destination_index = convert_coordinates_to_index(pair['destination'][0], pair['destination'][1], size_x=5)
    print(f"Source index: {source_index}, Destination index: {destination_index}")




Source index: 0, Destination index: 24
Source index: 13, Destination index: 6
